# import packages

In [1]:
from io import BytesIO
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
import pandas as pd

from influxdb import InfluxDBClient
from functools import reduce

import dcor
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.decomposition import PCA
import numpy as np
import pickle

pd.set_option('display.max_columns', 200)

In [2]:
# # Constants for the SharePoint connection
# TENANT_ID = "c9dc265f-a55d-466a-920c-9eb2e81f750f"
# N2_URL = "https://n2applied.sharepoint.com/sites/"
# APPLICATION_ID = "f83dae31-1ffa-4457-8e00-bc48c0f02a29"

# def authenticate_sharepoint(site_name: str):
#     """Authenticate and return a client context for a SharePoint site."""
#     site_url = f'{N2_URL}{site_name}'
#     ctx_auth = AuthenticationContext(url=site_url)
#     ctx_auth.with_interactive(TENANT_ID, APPLICATION_ID)
#     ctx = ClientContext(site_url, ctx_auth)
#     return ctx

# def download_file(ctx: ClientContext, folder_path: str, file_name: str):
#     """Download a file from SharePoint."""
#     file_path = f'{folder_path}/{file_name}'
#     with open(file_name, "wb") as file:
#         ctx.web.get_file_by_server_relative_url(file_path).download(file).execute_query()

# def load_excelsheet(ctx: ClientContext, folder_path: str, file_name: str, sheet_name: str) -> pd.DataFrame:
#     """Load an Excel sheet from SharePoint into a pandas DataFrame."""
#     file_path = f'{folder_path}/{file_name}'
#     excel_file = ctx.web.get_file_by_server_relative_url(file_path)
#     file_content = excel_file.get_content().execute_query()
#     excel_sheet = pd.read_excel(io=BytesIO(file_content.value), sheet_name=sheet_name, skiprows=8)
#     return excel_sheet

# if __name__ == "__main__":
#     # Fill in these variables with your specific values
#     site_name = 'RD'  # Replace with your actual site name
#     folder_path = "Projects/N2-PNG Stability and qualification"  # Replace with your actual folder path
#     file_name = "2024-1068-Testplan and log.xlsx"  # Replace with your actual file name
#     sheet_name = '2024-1018-TestLog'  # Replace with the sheet name you want
#     # Authenticate and obtain client context
#     ctx = authenticate_sharepoint(site_name)
#     # Download the file
#     download_file(ctx, folder_path, file_name)
#     # Load the Excel sheet into a DataFrame
#     excel_data = load_excelsheet(ctx, folder_path, file_name, sheet_name)

In [3]:
excel_data = pd.read_excel('2024-1068-Testplan and log.xlsx','2024-1018-TestLog', skiprows=8)
# excel_data

# Specify TB

In [4]:
tb_number = 13
lt = 30

# Influx data

## filter

In [5]:
## Rename labels
metadata_df = excel_data.rename(columns={'DSE material loss [g/h]':'material_loss_rate'})
metadata_df.columns
## Filter Excel Data
metadata_df = metadata_df[['Test_ID', 'Run Order', 'TB', 'Start Date [dd.mm.yyyy]',
       'Exp start Time [hh:mm]', 'Stop date\n [dd.mm.yyy]',
       'Exp stop Time [hh:mm]', 'Duration [h]', 'Quench diameter\n[mm]',
       'Quench distance\n[mm]', 'Running mode', 'Target Power [kW]',
       'Target current [A]', 'Target voltage [V]', 'Target aiflow [m3/h]',
       'Swirlinator type', 'Swirlinator inlets', 'Inlet diameter [mm]',
       'Coolant temp [C]', 'DSE ', 'DSE type', 'USE #',
       'DSE weight before [g]', 'DSE weight after [g]', 'material_loss_rate',
       'DSE material loss [g]', 'USE depth before [mm]',
       'USE depth after [mm]', 'USE material loss [mm/h]']].copy()

metadata_df['DSE material loss [g/h]'] = pd.to_numeric(metadata_df['material_loss_rate'], errors='coerce')

# dropped non valid rows
metadata_df.dropna(subset=['Test_ID',
                           'Exp start Time [hh:mm]',
                           'Start Date [dd.mm.yyyy]',
                           'Exp start Time [hh:mm]',
                           'Stop date\n [dd.mm.yyy]',
                           'Exp stop Time [hh:mm]',
                           'DSE material loss [g/h]'
                           ], inplace=True)

# Only include rows where Test OK = 1
# excel_data = excel_data.query('`Test OK 1=OK` == 1')
# excel_data = excel_data.query('`Test OK 1=OK` != 1')

metadata_df = metadata_df.query('`DSE ` == ["B52#1","B52#2","B52#3"]')

## Add Timestamps


In [6]:
# Convert columns to datetime and catch errors
metadata_df['exp_start'] = pd.to_datetime(metadata_df['Start Date [dd.mm.yyyy]'].astype(str) + ' ' + metadata_df['Exp start Time [hh:mm]'].astype(str), errors='coerce').dt.tz_localize('Europe/Oslo')
metadata_df['exp_stop'] = pd.to_datetime(metadata_df['Stop date\n [dd.mm.yyy]'].astype(str) + ' ' + metadata_df['Exp stop Time [hh:mm]'].astype(str), errors='coerce').dt.tz_localize('Europe/Oslo')

# Drop rows where datetime conversion failed (NaT values)
metadata_df = metadata_df.dropna(subset=['exp_start', 'exp_stop'])

C:\Users\Admin Henry Chen\AppData\Local\Temp\ipykernel_26764\3801585021.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  metadata_df['exp_start'] = pd.to_datetime(metadata_df['Start Date [dd.mm.yyyy]'].astype(str) + ' ' + metadata_df['Exp start Time [hh:mm]'].astype(str), errors='coerce').dt.tz_localize('Europe/Oslo')
C:\Users\Admin Henry Chen\AppData\Local\Temp\ipykernel_26764\3801585021.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  metadata_df['exp_stop'] = pd.to_datetime(metadata_df['Stop date\n [dd.mm.yyy]'].astype(str) + ' ' + metadata_df['Exp stop Time [hh:mm]'].astype(str), errors='coerce').dt.tz_localize('Europe/Oslo')


In [7]:
## Removing specific datapoints
metadata_df = metadata_df.query(f'TB == {tb_number}')
test_ids_to_exclude = []
metadata_df = metadata_df[~metadata_df['Test_ID'].isin(test_ids_to_exclude)]
metadata_df_copy = metadata_df.set_index('Test_ID').copy()
metadata_dict = metadata_df_copy.to_dict('index')
# This eliminates need for filtering in subsequent operations
print(f"antall exp = {len(metadata_dict.keys())}\nexcluded = {test_ids_to_exclude}")

antall exp = 5
excluded = []


## Sensor

In [8]:
## Sensor Data
# Constants
HOST = '192.168.1.3'
DATABASE = 'rnd'
SELECTED_MACHINE = f'TS{tb_number}'  # Avoid hardcoding by fetching dynamically or using config

# Use a context manager to handle the InfluxDB client connection
with InfluxDBClient(host=HOST, database=DATABASE) as client:
    # Fetch measurements and filter for the selected machine
    measurements = client.query('SHOW MEASUREMENTS')
    selected_machine = next((row['name'] for row in measurements.get_points() if row['name'] == SELECTED_MACHINE), None)

    if not selected_machine:
        raise ValueError(f"Machine '{SELECTED_MACHINE}' not found in measurements.")

    # Fetch equipment and data tag values in a single query
    tag_query = f'SHOW TAG VALUES FROM "{SELECTED_MACHINE}" WITH KEY IN ("equipment", "data")'
    tag_results = client.query(tag_query)

    # Extract equipment and data lists
    equipment_list = [row['value'] for row in tag_results.get_points() if row['key'] == 'equipment']
    data_list = [row['value'] for row in tag_results.get_points() if row['key'] == 'data']

# Print results
# print("Selected Machine:", selected_machine)
# print("Equipment List:", equipment_list)
# print("Data List:", data_list)

# for equipment in equipment_list:
#      for data in data_list:
#          print(equipment,data)

# Hardcoded equipment and data lists
f_equipment_list = ['G2101', 'FV2001', 'system', 'FT2301', 'TT2302', 'TT2308', 'FT2302']
f_data_list = ['arc_voltage', 'arc_power', 'arc_current', 'flow', 'pressure', 'running', 'temperature']

# Initialize a dictionary to store DataFrames for each Test ID
sensor_dict = {}

for Test_ID, excel in metadata_dict.items():
    exp_start = excel['exp_start']
    exp_stop = excel['exp_stop']
    
    # Initialize an empty list to store DataFrames for the current Test ID
    dataframes = []
    
    # Iterate over equipment and data lists
    for equipment in f_equipment_list:
        for data in f_data_list:
            query = f"""
            SELECT mean(value) AS mean_value
            FROM {selected_machine}
            WHERE equipment = '{equipment}' AND data = '{data}'
            AND time > '{exp_start.isoformat()}' AND time < '{exp_stop.isoformat()}'
            GROUP BY time({lt}s, {lt*(-1)}s)
            """
            data_points = client.query(query)
            data_rows = list(data_points.get_points())
            
            if data_rows:
                # Create a DataFrame for the current equipment and data
                temp_df = pd.DataFrame({
                    'timestamp': pd.to_datetime([dp['time'] for dp in data_rows]).tz_convert('Europe/Oslo'),
                    f'{equipment}({data})': [dp['mean_value'] for dp in data_rows]
                })

                # Append the DataFrame to the list
                dataframes.append(temp_df)
    
    # Merge all DataFrames for the current Test ID on the 'timestamp' column
    if dataframes:
        merged_sensor_df = reduce(lambda left, right: pd.merge(left, right, on='timestamp', how='outer'), dataframes)
        # merged_sensor_df.set_index('timestamp', inplace=True)
    else:
        merged_sensor_df = pd.DataFrame()

    merged_sensor_df = merged_sensor_df.interpolate()
    merged_sensor_df = merged_sensor_df.dropna()
    
    # Store the merged DataFrame in the dictionary with Test ID as the key
    sensor_dict[Test_ID] = merged_sensor_df

## handle missing numbers

In [9]:
print(merged_sensor_df.isna().sum().to_string())

timestamp              0
G2101(arc_voltage)     0
G2101(arc_power)       0
G2101(arc_current)     0
G2101(running)         0
G2101(temperature)     0
FV2001(flow)           0
FV2001(pressure)       0
FV2001(temperature)    0
system(running)        0
FT2301(flow)           0
FT2301(temperature)    0
TT2302(temperature)    0
TT2308(temperature)    0
FT2302(flow)           0
FT2302(temperature)    0


## Ftir

In [10]:
# ## FTIR Data
# # Constants
# HOST = '192.168.1.3'
# DATABASE = 'instruments'
# SELECTED_MACHINE = 'protea_ftir_2'  # Avoid hardcoding by fetching dynamically or using config

# # Use a context manager to handle the InfluxDB client connection
# with InfluxDBClient(host=HOST, database=DATABASE) as client:
#     # Fetch measurements and filter for the selected machine
#     measurements = client.query('SHOW MEASUREMENTS')
#     selected_machine = next((row['name'] for row in measurements.get_points() if row['name'] == SELECTED_MACHINE), None)

#     if not selected_machine:
#         raise ValueError(f"Machine '{SELECTED_MACHINE}' not found in measurements.")

#     # Fetch equipment and data tag values in a single query
#     tag_query = f'SHOW TAG VALUES FROM "{SELECTED_MACHINE}" WITH KEY IN ("equipment", "data")'
#     tag_results = client.query(tag_query)

#     # Extract equipment and data lists
#     equipment_list = [row['value'] for row in tag_results.get_points() if row['key'] == 'equipment']
#     data_list = [row['value'] for row in tag_results.get_points() if row['key'] == 'data']

# # # Print results
# # print("Selected Machine:", selected_machine)
# # print("Equipment List:", equipment_list)
# # print("Data List:", data_list)

# # for equipment in equipment_list:
# #      for data in data_list:
# #          print(equipment,data)

# # Hardcoded equipment and data lists
# f_equipment_list = ['Channel_2']
# f_data_list = ['NO_Corrected', 'NO2_Corrected']

# # Initialize a dictionary to store DataFrames for each Test ID
# ftir_dict = {}

# for Test_ID, excel in metadata_dict.items():
#     exp_start = excel['exp_start']
#     exp_stop = excel['exp_stop']
    
#     # Initialize an empty list to store DataFrames for the current Test ID
#     dataframes = []
    
#     # Iterate over equipment and data lists
#     for equipment in f_equipment_list:
#         for data in f_data_list:
#             query = f"""
#             SELECT mean(value) AS mean_value
#             FROM {selected_machine}
#             WHERE equipment = '{equipment}' AND data = '{data}'
#             AND time > '{exp_start.isoformat()}' AND time < '{exp_stop.isoformat()}'
#             GROUP BY time(30s, -30s)
#             """
#             data_points = client.query(query)
#             data_rows = list(data_points.get_points())
            
#             if data_rows:
#                 # Create a DataFrame for the current equipment and data
#                 temp_df = pd.DataFrame({
#                     'timestamp': pd.to_datetime([dp['time'] for dp in data_rows]).tz_convert('Europe/Oslo'),
#                     f'{equipment}({data})': [dp['mean_value'] for dp in data_rows]
#                 })

#                 # Append the DataFrame to the list
#                 dataframes.append(temp_df)
    
#     # Merge all DataFrames for the current Test ID on the 'timestamp' column
#     if dataframes:
#         merged_ftir_df = reduce(lambda left, right: pd.merge(left, right, on='timestamp', how='outer'), dataframes)
#         # merged_ftir_df.set_index('timestamp', inplace=True)
#     else:
#         merged_ftir_df = pd.DataFrame()
    
#     # Store the merged DataFrame in the dictionary with Test ID as the key
#     ftir_dict[Test_ID] = merged_ftir_df

# Manipulations


## Merge Sensor and FTIR Data

In [11]:
## Manipulations
# Check column names in sensor_dict
print(sensor_dict[Test_ID].columns)
# Check column names in ftir_dict
# print(ftir_dict[Test_ID].columns)

merged_dict = {}

for Test_ID in sensor_dict.keys():
    # Merge the DataFrames from both dictionaries on the 'timestamp' column
    # influx_df = pd.merge(sensor_dict[Test_ID], ftir_dict[Test_ID], on='timestamp', how='outer')
    influx_df = pd.DataFrame(sensor_dict[Test_ID])
    # influx_df = influx_df.query('`G2101(arc_power)` > 15 & `G2101(arc_voltage)` > 500 & `FV2001(flow)` > 25')
    influx_df = influx_df.query('`system(running)` == 1')

    # influx_df['NO/NO2'] = influx_df['Channel_2(NO_Corrected)'] / influx_df['Channel_2(NO2_Corrected)']
    # influx_df['NOx%'] = influx_df['Channel_2(NO_Corrected)'] + influx_df['Channel_2(NO2_Corrected)']
    # std_temp = 273
    # std_pressure = 101325
    # Mol_mas_N = 14
    # R = 8.3145
    # influx_df['EC_calc'] = (100000 * std_temp * R * influx_df['G2101(arc_power)']) / (std_pressure * Mol_mas_N * influx_df['FV2001(flow)'] * influx_df['NOx%'])
    
    influx_df['enthalpy_calc'] = influx_df['G2101(arc_power)'] / influx_df['FV2001(flow)']

    # Store the merged DataFrame in the dictionary with Test ID as the key
    merged_dict[Test_ID] = influx_df

merged_influx_df = pd.concat(merged_dict.values(), ignore_index=True)

Index(['timestamp', 'G2101(arc_voltage)', 'G2101(arc_power)',
       'G2101(arc_current)', 'G2101(running)', 'G2101(temperature)',
       'FV2001(flow)', 'FV2001(pressure)', 'FV2001(temperature)',
       'system(running)', 'FT2301(flow)', 'FT2301(temperature)',
       'TT2302(temperature)', 'TT2308(temperature)', 'FT2302(flow)',
       'FT2302(temperature)'],
      dtype='object')


## Create Statistical Dataframe


In [12]:
stat_dict = {}

# Iterate over the keys (Test_IDs)
for Test_ID in merged_dict.keys():
    # merged_df = pd.merge(merged_dict[Test_ID], ftir_dict[Test_ID], on='timestamp', how='outer')
    merged_df = merged_dict[Test_ID]
    # merged_df['NO/NO2']=merged_df['Channel_2(NO_Corrected)']/merged_df['Channel_2(NO2_Corrected)']
    # merged_df['NOx%']=merged_df['Channel_2(NO_Corrected)']+merged_df['Channel_2(NO2_Corrected)']
    
    # std_temp = 273
    # std_pressure = 101325
    # Mol_mas_N = 14
    # R = 8.3145
    # merged_df['EC_calc'] = (100000*std_temp*R*merged_df['G2101(arc_power)']) / (std_pressure*Mol_mas_N*merged_df['FV2001(flow)']*merged_df['NOx%'])
    merged_df['enthalpy_calc'] = merged_df['G2101(arc_power)'] / merged_df['FV2001(flow)']

    # Exclude the 'timestamp' column from calculations
    data_columns = merged_df.columns.difference(['timestamp'])
    mean_values = merged_df[data_columns].mean()
    std_values = merged_df[data_columns].std()
    cv_values = std_values / mean_values

    # Store the results in a DataFrame
    stats = {'mean': mean_values, 'std': std_values, 'cv': cv_values}
    stat_df = pd.DataFrame(stats).transpose()
    stat_df['Test_ID'] = Test_ID
    stat_dict[Test_ID] = stat_df

# Concatenate all DataFrames into one
all_stats_df = pd.concat(stat_dict.values(), ignore_index=False)

stat_df = pd.DataFrame()
# For each parameter, append mean, std, cv in order
for parameter in data_columns:
    stat_df = pd.concat(
        [stat_df, 
         all_stats_df.loc['mean', [parameter]].reset_index(drop=True).rename(columns={parameter: parameter + '_mean'}),
         all_stats_df.loc['std', [parameter]].reset_index(drop=True).rename(columns={parameter: parameter + '_std'}),
         all_stats_df.loc['cv', [parameter]].reset_index(drop=True).rename(columns={parameter: parameter + '_cv'})],
        axis=1
    )
# Add Test_ID as a column
stat_df['Test_ID'] = all_stats_df.iloc[::3, all_stats_df.columns.get_loc('Test_ID')].to_list()

print("Reformatted Statistics DataFrame:")
# stat_df.tail(10)
# print(stat_df.to_string(index=False))

Reformatted Statistics DataFrame:


## Merge Influx and excel


In [13]:
final_df = pd.merge(metadata_df_copy, stat_df, on='Test_ID')
# final_df = final_df.drop(columns=[col for col in final_df.columns if 'cv' in col])
# final_df.to_csv('final_df.csv')

# id = final_df[final_df['Test_ID']=='2025-exp30']
# print('exp 30 energy cost =', id['EC_calc_mean'].to_string(index=False))

# final_df.isna().sum()

# final_df.tail(5)

# save as pickles

In [14]:
merged_influx_df.to_pickle(f'TB{tb_number}_merged_influx_df.pkl')
stat_df.to_pickle(f'TB{tb_number}_stat_df.pkl')
final_df.to_pickle(f'TB{tb_number}_final_df.pkl')

with open(f'TB{tb_number}_merged_dict.pkl', 'wb') as file:
    pickle.dump(merged_dict, file)
# Save sensor_dict as a pickle file
with open(f'TB{tb_number}_sensor_dict.pkl', 'wb') as file:
    pickle.dump(sensor_dict, file)
# Save ftir_dict as a pickle file
# with open(f'TB{tb_number}_ftir_dict.pkl', 'wb') as file:
#     pickle.dump(ftir_dict, file)